In [3]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [4]:
# Load saved data
from google.colab import drive
drive.mount('/content/drive')
data = np.load('/content/drive/My Drive/Github/mtc-device-activation/data/activity-models/e2-general-model.npz', allow_pickle=True)
all_DeviceLocations = data['all_DeviceLocations']
num_samples = data['num_samples']
device_count = data['device_count']
sparsity = data['sparsity']

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-theta.npz", allow_pickle=True)
all_theta = data["all_theta"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-gamma.npz", allow_pickle=True)
all_gamma = data["all_gamma"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-z.npz", allow_pickle=True)
all_z = data["all_z"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-noise.npz", allow_pickle=True)
all_noise = data["all_noise"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-received-signal.npz", allow_pickle=True)
all_received_signal = data["all_received_signal"]

data = np.load("/content/drive/My Drive/Github/mtc-device-activation/data/communication-models/e2-communication-simulation-general.npz", allow_pickle=True)
snr_db_array = data["snr_db_array"]
pilot_length_array = data["pilot_length_array"]
M = data["M"]

snr_db_index = 2
snr_db=snr_db_array[snr_db_index]

pilot_length_index = 2
pilot_length=pilot_length_array[pilot_length_index]

max_iter = 500

Mounted at /content/drive


In [5]:
num_samples = 1000
sample_range = range(0, 1000)

Correlation-Aware Sparse Bayesian Learning (CA-SBL)

In [6]:
def casbl_anc_convergence(z, gamma, theta, y, noise_var, loc, alpha=1.00, beta=0.1, rho=7, U=20, max_iter=500, stopping_criterion=1e-4):
    """Correlation Aware Sparse Bayesian Learning (CASBL) for Multiple Measurement Vectors (MMV)."""

    M = y.shape[1]  # Number of antennas (columns of y)

    L = theta.shape[0]  # Pilot Length (rows of theta)
    N = theta.shape[1] # Number of MTDs (columns of theta)

    nmse_list = []

    # Initialize Gamma (covariance matrix of the signal)
    Gamma = np.eye(N) * 0.1  # Initial guess

    if rho == 0:
        correlation_matrix = np.eye(N)  # Identity matrix when no correlation
    else:
        # Compute pairwise distances (N x N matrix)
        distance_matrix = np.linalg.norm(loc[:, np.newaxis, :] - loc[np.newaxis, :, :], axis=2)

        # Compute exponential decay matrix
        correlation_matrix = np.maximum((np.exp(-distance_matrix / rho) - np.exp(-U / rho)) / (1 - np.exp(-U / rho)),0)

    # Outer loop over the maximum number of iterations
    for t in range(max_iter):
        # E-Step: Compute the mean (mu_x) and covariance (Sigma_x) of x
        Sigma_y = theta @ Gamma @ theta.conj().T + noise_var * np.eye(L)
        Sigma_y_inv = np.linalg.inv(Sigma_y)
        Sigma_x = Gamma - Gamma @ theta.conj().T @ Sigma_y_inv @ theta @ Gamma
        mu_x = Gamma @ theta.conj().T @ Sigma_y_inv @ y

        # Compute Q
        Q = (np.linalg.norm(mu_x, axis=1) ** 2) / M + np.diag(Sigma_x)

        # Compute P
        P = 2 * alpha * (beta - correlation_matrix) @ np.diag(Gamma)
        P = np.maximum(P, 1e-8)

        # Compute gamma update
        gamma_new = (np.sqrt(1 + 4 * P * Q) - 1) / (2 * P)

        # Ensure non-negativity (numerical stability)
        gamma_new = np.clip(np.real(gamma_new), 0, 1)

        # Update Gamma
        Gamma_old = np.diag(Gamma)
        Gamma = np.diagflat(gamma_new)

        #Gamma_conv = np.where(Gamma < 0.05, 0, 1)

        #Sigma_y_conv = theta @ Gamma_conv @ theta.conj().T + noise_var * np.eye(L)
        #Sigma_y_inv_conv = np.linalg.inv(Sigma_y_conv)
        #mu_x_conv = Gamma_conv @ theta.conj().T @ Sigma_y_inv_conv @ y

        # Compute NMSE
        nmse = np.linalg.norm(z[gamma == 1] - mu_x[gamma == 1])**2 / np.linalg.norm(z[gamma == 1])**2
        nmse_list.append(nmse)

        # Check for convergence
        if np.linalg.norm(Gamma_old - gamma_new) < stopping_criterion:
            print(f"Converged after {t+1} outer iterations")
            break

    return nmse_list


In [7]:
# Placeholder arrays for results
nmse_iter_res = np.zeros((num_samples, max_iter))

In [8]:
# Loop through each sample with an outer progress bar
for sample_index in tqdm(sample_range, desc="Processing Samples", position=0):
    gamma = all_gamma[sample_index]  # Ground truth gamma values (true support)
    loc = all_DeviceLocations[sample_index]
    theta = all_theta[sample_index, pilot_length_index, :pilot_length]
    received_signal = all_received_signal[sample_index, pilot_length_index, snr_db_index, :pilot_length]
    z = all_z[sample_index]
    snr = 10 ** (snr_db / 10)
    signal_power = np.mean(np.abs(np.matmul(theta, z))**2)
    noise_power = signal_power / snr

    # Apply SBL algorithm for the current sample and Pilot Length
    nmse_per_iter = casbl_anc_convergence(z, gamma, theta, received_signal, noise_power, loc)

    # Save NMSEs (pad if early stopping)
    nmse_iter_res[sample_index, :len(nmse_per_iter)] = nmse_per_iter

Processing Samples:   3%|▎         | 32/1000 [05:25<2:43:02, 10.11s/it]

Converged after 486 outer iterations


Processing Samples:   4%|▎         | 37/1000 [06:14<2:38:34,  9.88s/it]

Converged after 448 outer iterations


Processing Samples:   4%|▍         | 44/1000 [07:24<2:36:34,  9.83s/it]

...

Processing Samples:  96%|█████████▌| 959/1000 [2:41:04<06:51, 10.03s/it]

Converged after 478 outer iterations


Processing Samples:  98%|█████████▊| 985/1000 [2:45:26<02:30, 10.05s/it]

Converged after 491 outer iterations


Processing Samples: 100%|██████████| 1000/1000 [2:47:58<00:00, 10.08s/it]


In [9]:
np.savez_compressed(
    '/content/drive/My Drive/Github/mtc-device-activation/data/results/e2-results-1000-convergence-casbl-anc-alpha-1-00-beta-0-10-snr-db-12-pilot-length-30.npz',
    nmse_iter_res=nmse_iter_res,
    pilot_length=pilot_length,
    snr_db=snr_db
)

print("All results have been saved")

All results have been saved
